In [155]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")
print(f"Analysis run on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 157, Finished, Available, Finished)

Libraries imported successfully!
Analysis run on: 2025-07-27 00:04:55


In [156]:
# Check whether running in Fabric or locally, and set the data location accordingly
if "AZURE_SERVICE" in os.environ:
    is_fabric = True
    data_location = "abfss://7e373771-c704-4855-bb94-026ffb6be497@onelake.dfs.fabric.microsoft.com/740e989a-d750-4fd9-a4d9-def5fe22a5db/Files/forecasting/"
    print("Running in Fabric, setting data location to /lakehouse/default/Files/")
else:
    is_fabric = False
    data_location = ""
    print("Running locally, setting data location to current directory")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 158, Finished, Available, Finished)

Running in Fabric, setting data location to /lakehouse/default/Files/


In [157]:
# Load the combined sales economic data
data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_economic_and_future.csv')
# data = pd.read_csv(data_location + 'modelGeneratedData/overall_monthly_with_trend_seasonality_economic_and_future.csv')
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

print("=== DATASET OVERVIEW ===")
print(f"Dataset shape: {data.shape}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"\nData types:")
print(data.dtypes)
print(f"\nFirst few rows:")
data.head()

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 159, Finished, Available, Finished)

=== DATASET OVERVIEW ===
Dataset shape: (121, 57)
Date range: 2015-01-01 00:00:00 to 2025-01-01 00:00:00

Data types:
Date                                  datetime64[ns]
Quantity Invoiced                              int64
Quantity Invoiced Mean                       float64
Quantity Invoiced Count                        int64
Unique Customers                               int64
Unique Products                                int64
Unique Categories                              int64
Unique SubCategories                           int64
Unique EndMarkets L1                           int64
Unique EndMarkets L2                           int64
future_orders_count                            int64
future_orders_qty_total                        int64
future_orders_qty_next_1m                      int64
future_orders_qty_next_3m                      int64
future_orders_qty_next_6m                      int64
future_orders_qty_next_12m                     int64
future_orders_avg_lead_time       

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Unique Customers,Unique Products,Unique Categories,Unique SubCategories,Unique EndMarkets L1,Unique EndMarkets L2,...,data_Factory_Utilization,data_Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6)
0,2015-01-01,22725114,68039.26347,334,160,53,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1
1,2015-02-01,23032809,63276.94780,364,183,52,4,15,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3
2,2015-03-01,27527951,76679.52925,359,180,50,4,12,3,5,...,0.4302,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5
3,2015-04-01,25864804,68789.37234,376,177,58,4,15,3,6,...,0.4372,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4
4,2015-05-01,22517479,77379.65292,291,146,51,4,14,3,5,...,0.4381,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1


### Importing New Data - Feature Engineering

1. FMCG_Production_UTF8.csv (Overall Monthly Average)

In [158]:
# # FMCG_Production_UTF8.csv (Overall Monthly Average)

# import pandas as pd

# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/FMCG_Production_UTF8.csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/FMCG_Production_UTF8.csv")

# # Converting 'Date' columns to datetime format for both datasets
# df['Date'] = pd.to_datetime(df['Date'], errors='coerce')  # Automatically infer format
# data['Date'] = pd.to_datetime(data['Date'], errors='coerce') 

# # Converting 'Production Index' to numeric (force non-numeric values to NaN)
# df['Production Index'] = pd.to_numeric(df['Production Index'], errors='coerce')

# # Ensuring 'Country' exists and the 'Date' column is set to datetime
# # Add a 'Month' column to group by month
# df['Month'] = df['Date'].dt.to_period('M')  

# # Step 1: Calculate the monthly average for each country
# monthly_avg_per_country = df.groupby(['Country', 'Month'], as_index=False)['Production Index'].mean()

# # Step 2: Calculate the overall monthly average for all countries
# monthly_avg_all_countries = monthly_avg_per_country.groupby('Month', as_index=False)['Production Index'].mean()

# # Step 3: Merge the overall monthly average into the original data
# data = pd.merge(data, monthly_avg_all_countries, left_on=data['Date'].dt.to_period('M'), right_on='Month', how='left')

# # Filling any remaining missing values: forward > backward (if necessary)
# data['Production Index'] = data['Production Index'].fillna(method='ffill').fillna(method='bfill')

# # Final check for missing values
# print("Null values after cleaning:\n", data.isnull().sum())

# display(data.head())

# data


StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 160, Finished, Available, Finished)

2. FMCG - Production Index (Germany).csv

In [159]:
# import pandas as pd
# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/FMCG - Production Index (Germany).csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/FMCG - Production Index (Germany).csv")
# display(df)

# # Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])

# # Normalize both dates to first of the month (align monthly frequency)
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # Confirm it's loaded correctly
# print(df.head())

# # Merge the new data into your original data
# data = pd.merge(data, df[['Date', 'Production Index']], on='Date', how='left')


# # Forward fill missing data values
# data['Production Index'] = data['Production Index'].fillna(method='ffill').fillna(method='bfill')

# # Verify there are no missing values
# print(data.isnull().sum())

# # Check if the merge was successful
# print(data.head())

# data

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 161, Finished, Available, Finished)

3. Baltic Dry Index Historical Results Price Data.csv

In [160]:
# # import pandas as pd
# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Baltic Dry Index Historical Results Price Data.csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Baltic Dry Index Historical Results Price Data.csv")
# display(df)


# # Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=False)


# # Normalize both dates to first of the month (align monthly frequency)
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()


# # Confirm it's loaded correctly
# print(df.head())

# # Merge the new data into your original data
# data = pd.merge(data, df[['Date', 'BDI_Closing_Value']], on='Date', how='left')

# # Forward fill missing data values
# data['BDI_Closing_Value'] = data['BDI_Closing_Value'].fillna(method='ffill')

# # Verify there are no missing values
# print(data.isnull().sum())

# # Check if the merge was successful
# print(data.head())

# data

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 162, Finished, Available, Finished)

4. Economic Activity.csv

In [161]:
import pandas as pd
# Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Economic Activity.csv"
df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Economic Activity.csv")
display(df)

# Convert the 'Date' column to datetime format to ensure compatibility
df['Date'] = pd.to_datetime(df['Date'])

# Normalize both dates to first of the month (align monthly frequency)
df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# Confirm it's loaded correctly
print(df.head())

# Merge the new data into your original data
data = pd.merge(data, df[['Date', 'Economic Activity']], on='Date', how='left')

# Forward fill missing data values
data['Economic Activity'] = data['Economic Activity'].fillna(method='ffill')

# Verify there are no missing values
print(data.isnull().sum())

# Check if the merge was successful
print(data.head())

data


StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 163, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 52e631b8-47aa-4d26-a39c-b9b534ba7272)

        Date  Economic Activity
0 1955-01-01           23.94956
1 1955-02-01           25.36325
2 1955-03-01           23.61693
3 1955-04-01           25.61273
4 1955-05-01           25.36325
Date                                  0
Quantity Invoiced                     0
Quantity Invoiced Mean                0
Quantity Invoiced Count               0
Unique Customers                      0
Unique Products                       0
Unique Categories                     0
Unique SubCategories                  0
Unique EndMarkets L1                  0
Unique EndMarkets L2                  0
future_orders_count                   0
future_orders_qty_total               0
future_orders_qty_next_1m             0
future_orders_qty_next_3m             0
future_orders_qty_next_6m             0
future_orders_qty_next_12m            0
future_orders_avg_lead_time           0
future_orders_min_lead_time           0
future_orders_max_lead_time           0
future_orders_due_next_month          0
future_o

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Unique Customers,Unique Products,Unique Categories,Unique SubCategories,Unique EndMarkets L1,Unique EndMarkets L2,...,data_Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6),Economic Activity
0,2015-01-01,22725114,68039.26347,334,160,53,4,15,3,5,...,76.7556,0.2276,0.2302,14.4600,15.2300,-0.50,-0.39,86.8,76.1,98.50821
1,2015-02-01,23032809,63276.94780,364,183,52,4,15,3,5,...,76.7556,0.2276,0.2302,14.4600,15.2300,-0.32,-0.50,98.8,89.3,98.60822
2,2015-03-01,27527951,76679.52925,359,180,50,4,12,3,5,...,76.7556,0.2276,0.2302,14.4600,15.2300,-0.38,-0.32,90.0,91.5,98.30819
3,2015-04-01,25864804,68789.37234,376,177,58,4,15,3,6,...,76.8014,0.2276,0.2302,14.4600,15.2300,-0.35,-0.38,83.3,87.4,99.00825
4,2015-05-01,22517479,77379.65292,291,146,51,4,14,3,5,...,76.8657,0.2276,0.2302,14.4600,15.2300,-0.54,-0.35,98.0,88.1,100.20840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2024-09-01,27795162,101813.78020,273,130,49,5,12,3,5,...,76.5696,0.3233,0.3244,28.6451,25.1613,0.10,0.20,89.6,83.9,112.30940
117,2024-10-01,26484363,92280.01045,287,135,53,5,15,3,6,...,76.0033,0.3233,0.3244,28.6451,25.1613,-0.34,0.10,84.8,79.6,112.30940
118,2024-11-01,24818685,114371.82030,217,114,46,4,12,3,6,...,76.0663,0.3233,0.3244,28.6451,25.1613,-0.28,-0.34,86.4,87.3,112.30940
119,2024-12-01,15512968,91252.75294,170,92,38,5,12,3,5,...,76.3797,0.3233,0.3244,28.6451,25.1613,-0.26,-0.28,86.0,89.2,112.30940


5. HICP (retest with this new model )

In [162]:
# import pandas as pd
# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/hicp_data.csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/Germany Economic data/hicp_data.csv")
# display(df)

# # Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])

# # Confirm it's loaded correctly
# print(df.head())

# # Merge the unemployment rate data into your original data
# data = pd.merge(data, df[['Date', 'HICP']], on='Date', how='left')

# # Forward fill missing unemployment rate values
# data['HICP'] = data['HICP'].fillna(method='ffill')

# # Verify there are no missing values
# print(data.isnull().sum())

# # Check if the merge was successful
# print(data.head())

# data



StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 164, Finished, Available, Finished)

6. Consumer Confidence Indicator - European Countries.csv (Germany Only)




In [163]:
# import pandas as pd
# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Consumer Confidence Indicator - European Countries .csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Consumer Confidence Indicator - European Countries .csv")
# display(df)

# # Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])

# # Normalize both dates to first of the month (align monthly frequency)
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # Confirm it's loaded correctly
# print(df.head())

# # Merge the new data into your original data
# data = pd.merge(data, df[['Date', 'Consumer Confidence Indicator (Germany Only)']], on='Date', how='left')

# # Forward fill missing data values
# data['Consumer Confidence Indicator (Germany Only)'] = data['Consumer Confidence Indicator (Germany Only)'].fillna(method='ffill')

# # Verify there are no missing values
# print(data.isnull().sum())

# # Check if the merge was successful
# print(data.head())

# data


StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 165, Finished, Available, Finished)

7. Consumer Confidence Indicator - European Countries.csv (All Countries)

In [164]:
# import pandas as pd
# # Load data into pandas DataFrame from "/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Consumer Confidence Indicator - European Countries .csv"
# df = pd.read_csv("/lakehouse/default/Files/forecasting/userProvidedData/dataExploration/Consumer Confidence Indicator - European Countries .csv")
# display(df)

# # Convert the 'Date' column to datetime format to ensure compatibility
# df['Date'] = pd.to_datetime(df['Date'])

# # Normalize both dates to first of the month (align monthly frequency)
# df['Date'] = df['Date'].dt.to_period('M').dt.to_timestamp()
# data['Date'] = data['Date'].dt.to_period('M').dt.to_timestamp()

# # Confirm it's loaded correctly
# print(df.head())

# # Merge the new data into your original data
# data = pd.merge(data, df[['Date', 'Consumer Confidence Indicator (Total)']], on='Date', how='left')

# # Forward fill missing data values
# data['Consumer Confidence Indicator (Total)'] = data['Consumer Confidence Indicator (Total)'].fillna(method='ffill')

# # Verify there are no missing values
# print(data.isnull().sum())

# # Check if the merge was successful
# print(data.head())


StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 166, Finished, Available, Finished)

## Data Preparation for Overall Forecasting

We'll prepare the data for overall quantity forecasting by aggregating all sales data across segments and subcategories to create a single time series for the total quantity.

In [165]:
# Create overall aggregation
print("=== CREATING OVERALL AGGREGATION ===")

# Print column names
print(f"Columns in dataset: {data.columns.tolist()}")

# Rename columns in the data
data = data.rename(columns={
    'data_PP_Spot': 'PP_Spot',
    'data_Resin': 'Resin',
    'data_WTI_Crude_Oil': 'WTI_Crude_Oil',
    'data_Natural_Gas': 'Natural_Gas',
    'data_Energy_Average': 'Energy_Average',
    'data_PPI_Freight': 'PPI_Freight',
    'data_PMI_Data': 'PMI_Data',
    'data_Factory_Utilization': 'Factory_Utilization',
    'data_Capacity_Utilization': 'Capacity_Utilization',
    'data_Beverage': 'Beverage',
    'data_Household_consumption': 'Household_consumption',
    'data_packaging': 'packaging',
    'data_Diesel': 'Diesel',
    'data_PPI_Delivery': 'PPI_Delivery',
    'data_Oil-to-resin': 'Oil-to-resin',
    'Electricity Price': 'Electricity Price',
    'Electricity Price (Lag6)': 'Electricity Price (Lag6)',
    'Gas Price': 'Gas Price',
    'Gas Price (Lag6)': 'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index': 'Global Supply Chain Pressure Index',
    'GSCPI (Lag1)': 'GSCPI (Lag1)',
    'Manufacturing Orders Volume Index': 'Manufacturing Orders Volume Index',
    'MOVI (Lag6)': 'MOVI (Lag6)',
    'packaging (Lag2)': 'packaging (Lag2)',
    'PPI_Freight (Lag2)': 'PPI_Freight (Lag2)',
    'trend': 'decomp_trend',
    'seasonality': 'decomp_seasonality',
    # Feature Engineering
    # 'Production Index':'Production in Industry (European)' #1
    # 'Production Index':'Consumer Goods (Germany)' #2
    # 'BDI_Closing_Value': 'BDI Closing Value' #3
    'Economic Activity': 'Economic Activity' #4
    # 'HICP':'HICP' #5
    # 'Consumer Confidence Indicator (Germany Only)': 'Consumer Confidence Indicator (Germany Only)' #6
    # 'Consumer Confidence Indicator (Total)': 'Consumer Confidence Indicator (Total)' #7
})

# Define exogenous variables for modeling
exog_vars = [
    'PP_Spot',
    'Resin',
    'PMI_Data',
    'Natural_Gas',
    # 'WTI_Crude_Oil',
    #'Factory_Utilization',
    'packaging',
    'Energy_Average',
    'Electricity Price (Lag6)',
    'Gas Price (Lag6)',
    'Global Supply Chain Pressure Index', # neutral
    # 'future_orders_qty_total',
    'future_orders_qty_next_1m',
    'future_orders_qty_next_3m',
    #'future_orders_qty_next_6m',
    # 'future_orders_qty_next_12m',
    # 'future_orders_avg_lead_time',
    # 'future_orders_min_lead_time',
    # 'future_orders_max_lead_time',
    # 'decomp_trend', 
    #'decomp_seasonality'

    # Feature Engineering
    # 'Production in Industry (European)' #1
    # 'Consumer Goods (Germany)'  #2
    # 'BDI Closing Value' #3
    'Economic Activity' #4
    # 'HICP' #5
    # 'Consumer Confidence Indicator (Germany Only)' #6
    # 'Consumer Confidence Indicator (Total)' #7
]

# Overall aggregation (sum across all segments and subcategories)
print(f"Overall time series shape: {data.shape}")

# Display summary statistics
print("\n=== OVERALL SUMMARY ===")
print(f"Overall total quantity range: {data['Quantity Invoiced'].min():,.0f} - {data['Quantity Invoiced'].max():,.0f}")
print(f"Date range: {data['Date'].min()} to {data['Date'].max()}")
print(f"Number of data points: {len(data)}")

# Display columns and their data types
print("\n=== COLUMNS AND DATA TYPES ===")
print(data.dtypes)

data.head()

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 167, Finished, Available, Finished)

=== CREATING OVERALL AGGREGATION ===
Columns in dataset: ['Date', 'Quantity Invoiced', 'Quantity Invoiced Mean', 'Quantity Invoiced Count', 'Unique Customers', 'Unique Products', 'Unique Categories', 'Unique SubCategories', 'Unique EndMarkets L1', 'Unique EndMarkets L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'qty_rolling_avg_12m', 'future_to_rolling_12m_ratio', 'has_future_orders', 'high_future_orders', 'future_customer_diversity_ratio', 'future_product_diversity_ratio', 'data_PP_Spot', 'data_Resin', 'data_WTI_Crude_Oil', 'data_Natural_Gas', 'data_Oil-to-resin

,Date,Quantity Invoiced,Quantity Invoiced Mean,Quantity Invoiced Count,Unique Customers,Unique Products,Unique Categories,Unique SubCategories,Unique EndMarkets L1,Unique EndMarkets L2,...,Capacity_Utilization,Electricity Price,Electricity Price (Lag6),Gas Price,Gas Price (Lag6),Global Supply Chain Pressure Index,GSCPI (Lag1),Manufacturing Orders Volume Index,MOVI (Lag6),Economic Activity
0,2015-01-01,22725114,68039.26347,334,160,53,4,15,3,5,...,76.7556,0.2276,0.2302,14.46,15.23,-0.50,-0.39,86.8,76.1,98.50821
1,2015-02-01,23032809,63276.94780,364,183,52,4,15,3,5,...,76.7556,0.2276,0.2302,14.46,15.23,-0.32,-0.50,98.8,89.3,98.60822
2,2015-03-01,27527951,76679.52925,359,180,50,4,12,3,5,...,76.7556,0.2276,0.2302,14.46,15.23,-0.38,-0.32,90.0,91.5,98.30819
3,2015-04-01,25864804,68789.37234,376,177,58,4,15,3,6,...,76.8014,0.2276,0.2302,14.46,15.23,-0.35,-0.38,83.3,87.4,99.00825
4,2015-05-01,22517479,77379.65292,291,146,51,4,14,3,5,...,76.8657,0.2276,0.2302,14.46,15.23,-0.54,-0.35,98.0,88.1,100.20840


In [166]:
# Visualize the overall time series
fig = go.Figure()

# Plot Overall Total Quantity
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Quantity Invoiced'],
              mode='lines+markers', name='Overall Total Quantity',
              line=dict(color='darkblue', width=3),
              marker=dict(size=6))
)

fig.update_layout(
    height=600,
    title_text="Overall Sales Quantity Time Series Analysis",
    xaxis_title="Date",
    yaxis_title="Quantity Invoiced",
    showlegend=True,
    template="plotly_white"
)

fig.show()

# Statistical summary
print("\n=== STATISTICAL SUMMARY ===")
print("Overall Quantity Statistics:")
print(data['Quantity Invoiced'].describe())

# Additional time series analysis
print(f"\nTime Series Characteristics:")
print(f"Mean: {data['Quantity Invoiced'].mean():,.0f}")
print(f"Standard Deviation: {data['Quantity Invoiced'].std():,.0f}")
print(f"Coefficient of Variation: {(data['Quantity Invoiced'].std() / data['Quantity Invoiced'].mean() * 100):.2f}%")

# Check for trends and seasonality
data_monthly = data.set_index('Date')['Quantity Invoiced']
monthly_avg = data_monthly.groupby(data_monthly.index.month).mean()
print(f"\nMonthly Averages:")
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
for month, avg in monthly_avg.items():
    print(f"  {month_names[month-1]}: {avg:,.0f}")

# Year-over-year growth
yearly_avg = data_monthly.groupby(data_monthly.index.year).mean()
print(f"\nYearly Averages:")
for year, avg in yearly_avg.items():
    print(f"  {year}: {avg:,.0f}")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 168, Finished, Available, Finished)


=== STATISTICAL SUMMARY ===
Overall Quantity Statistics:
count    1.210000e+02
mean     2.569414e+07
std      3.963701e+06
min      1.256533e+07
25%      2.303281e+07
50%      2.586480e+07
75%      2.823139e+07
max      3.352502e+07
Name: Quantity Invoiced, dtype: float64

Time Series Characteristics:
Mean: 25,694,142
Standard Deviation: 3,963,701
Coefficient of Variation: 15.43%

Monthly Averages:
  Jan: 26,139,639
  Feb: 25,506,276
  Mar: 28,644,908
  Apr: 24,628,411
  May: 25,084,145
  Jun: 27,862,406
  Jul: 27,817,574
  Aug: 26,186,554
  Sep: 28,143,352
  Oct: 25,329,172
  Nov: 24,806,704
  Dec: 18,136,016

Yearly Averages:
  2015: 24,146,230
  2016: 24,500,382
  2017: 26,040,790
  2018: 25,449,790
  2019: 28,086,842
  2020: 27,556,023
  2021: 28,535,274
  2022: 26,028,150
  2023: 22,195,984
  2024: 24,418,603
  2025: 25,494,397


# Overall Quantity Forecasting Implementation

Functions defining the forecasting methods for overall quantity prediction.

## Forecasting Models

We'll implement multiple forecasting approaches:
1. **ARIMA** - Auto-regressive Integrated Moving Average
2. **SARIMA** - Seasonal ARIMA with economic indicators
3. **Exponential Smoothing** - Holt-Winters method
4. **Prophet** - Meta's time series forecasting tool with trend and seasonality
5. **Ensemble** - Weighted combination of methods

In [167]:
def forecast_arima(series, steps=12, order=(1,1,1)):
    """
    ARIMA forecasting with automatic order selection if needed
    """
    try:
        model = ARIMA(series, order=order)
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=steps)
        conf_int = fitted_model.get_forecast(steps=steps).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Try simpler model if original fails
        try:
            model = ARIMA(series, order=(1,0,1))
            fitted_model = model.fit()
            forecast = fitted_model.forecast(steps=steps)
            conf_int = fitted_model.get_forecast(steps=steps).conf_int()
            return forecast, conf_int, fitted_model.aic
        except:
            # Last resort - simple naive forecast
            last_value = series.iloc[-1]
            forecast = pd.Series([last_value] * steps)
            conf_int = pd.DataFrame({
                'lower Quantity Invoiced': forecast * 0.9,
                'upper Quantity Invoiced': forecast * 1.1
            })
            return forecast, conf_int, float('inf')

def forecast_sarima(series, steps=12, exog=None, order=(1,1,1), seasonal_order=(1,1,1,12)):
    """
    SARIMA forecasting with external regressors
    """
    try:
        model = SARIMAX(series, exog=exog, order=order, seasonal_order=seasonal_order)
        fitted_model = model.fit(disp=False)
        
        # For forecast, we need future exogenous variables
        # Use last known values as a simple assumption
        if exog is not None:
            future_exog = pd.DataFrame([exog.iloc[-1]] * steps)
            future_exog.index = pd.date_range(start=exog.index[-1] + pd.DateOffset(months=1), periods=steps, freq='MS')
        else:
            future_exog = None
            
        forecast = fitted_model.forecast(steps=steps, exog=future_exog)
        conf_int = fitted_model.get_forecast(steps=steps, exog=future_exog).conf_int()
        return forecast, conf_int, fitted_model.aic
    except:
        # Fallback to simple ARIMA
        return forecast_arima(series, steps, order)

def forecast_exponential_smoothing(series, steps=12, seasonal_periods=12):
    """
    Exponential Smoothing (Holt-Winters) forecasting
    """
    try:
        if len(series) >= 2 * seasonal_periods:
            model = ExponentialSmoothing(series, trend='add', seasonal='add', seasonal_periods=seasonal_periods)
        else:
            model = ExponentialSmoothing(series, trend='add', seasonal=None)
        
        fitted_model = model.fit()
        forecast = fitted_model.forecast(steps=steps)
        
        # Simple confidence intervals based on residuals
        residuals = fitted_model.resid
        std_resid = residuals.std()
        conf_int = pd.DataFrame({
            'lower Quantity Invoiced': forecast - 1.96 * std_resid,
            'upper Quantity Invoiced': forecast + 1.96 * std_resid
        })
        
        return forecast, conf_int, fitted_model.aic
    except:
        # Fallback to ARIMA
        return forecast_arima(series, steps)

def forecast_prophet(series, steps=12, exog=None):
    """
    Prophet forecasting with trend, seasonality, and external regressors
    """
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    prophet_data = pd.DataFrame({
        'ds': series.index,
        'y': series.values
    })
    
    # Initialize Prophet model
    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=False,  # Monthly data doesn't need weekly seasonality
        daily_seasonality=False,   # Monthly data doesn't need daily seasonality
        seasonality_mode='multiplicative',
        changepoint_prior_scale=0.05,  # Controls flexibility of trend
        seasonality_prior_scale=10.0,  # Controls flexibility of seasonality
        interval_width=0.95
    )
    
    # Add external regressors if provided
    if exog is not None and len(exog.columns) > 0:
        # Add each regressor to the model
        for col in exog.columns:
            if col in ['PP_Spot', 'Resin', 'WTI_Crude_Oil', 'Natural_Gas', 'Energy_Average']:
                # Energy-related indicators tend to have strong impact
                model.add_regressor(col, prior_scale=0.5, mode='multiplicative')
            elif col in ['PMI_Data', 'Factory_Utilization', 'Capacity_Utilization']:
                # Manufacturing indicators
                model.add_regressor(col, prior_scale=0.3, mode='additive')
            else:
                # Other economic indicators
                model.add_regressor(col, prior_scale=0.1, mode='additive')
        
        # Add regressors to prophet_data
        for col in exog.columns:
            prophet_data[col] = exog[col].values
    
    # Fit the model
    model.fit(prophet_data)
    
    # Create future dataframe
    last_date = series.index[-1]
    future_dates = pd.date_range(
        start=last_date + pd.DateOffset(months=1), 
        periods=steps, 
        freq='MS'
    )
    
    future_df = pd.DataFrame({'ds': future_dates})
    
    # Add future regressor values if available
    if exog is not None and len(exog.columns) > 0:
        # Use last known values for future regressors (simple assumption)
        last_regressor_values = exog.iloc[-1]
        for col in exog.columns:
            future_df[col] = last_regressor_values[col]
    
    # Make forecast
    forecast_df = model.predict(future_df)
    
    # Extract forecast values and confidence intervals
    forecast = pd.Series(
        forecast_df['yhat'].values, 
        index=future_dates,
        name='Quantity Invoiced'
    )
    
    conf_int = pd.DataFrame({
        'lower Quantity Invoiced': forecast_df['yhat_lower'].values,
        'upper Quantity Invoiced': forecast_df['yhat_upper'].values
    }, index=future_dates)
    
    # Calculate approximate AIC using cross-validation or residual-based metric
    # Prophet doesn't have built-in AIC, so we'll use MAE on in-sample predictions
    in_sample_forecast = model.predict(prophet_data)
    mae = mean_absolute_error(prophet_data['y'], in_sample_forecast['yhat'])
    pseudo_aic = 2 * len(prophet_data) + 2 * np.log(mae)  # Approximation
    
    return forecast, conf_int, pseudo_aic

def ensemble_forecast(forecasts, aics=None):
    """
    Create ensemble forecast from multiple methods (weighted by inverse AIC)
    """
    weights = []

    if aics is None:
        weights = [1/len(forecasts)] * len(forecasts)
    else:
        weights = [1/aic if aic != float('inf') else 0 for aic in aics]
        total_weight = sum(weights)
        if total_weight > 0:
            weights = [w/total_weight for w in weights]
        else:
            # Equal weights if all AICs are infinite
            weights = [1/len(forecasts)] * len(forecasts)

    # If forecast index is not aligned, reindex to the first forecast's index
    first_index = forecasts[0].index
    for i in range(len(forecasts)):
        if not forecasts[i].index.equals(first_index):
            forecasts[i] = forecasts[i].reindex(first_index)

    # Print weights
    print(f"Model weights - ARIMA: {weights[0]:.3f}, SARIMA: {weights[1]:.3f}, EXP: {weights[2]:.3f}, Prophet: {weights[3]:.3f}")
    
    ensemble = sum(f * w for f, w in zip(forecasts, weights))

    return ensemble

print("Forecasting functions defined successfully!")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 169, Finished, Available, Finished)

Forecasting functions defined successfully!


## Calculate Accuracy Metrics

In [168]:
def calculate_accuracy_metrics(actual, predicted, method_name):
    """Calculate comprehensive accuracy metrics"""
    # Remove any NaN values
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    actual_clean = actual[mask]
    predicted_clean = predicted[mask]
    
    if len(actual_clean) == 0:
        return None
    
    # Calculate metrics
    mae = mean_absolute_error(actual_clean, predicted_clean)
    mse = mean_squared_error(actual_clean, predicted_clean)
    rmse = np.sqrt(mse)
    
    # Avoid division by zero for MAPE
    mape = np.mean(np.abs((actual_clean - predicted_clean) / np.where(actual_clean != 0, actual_clean, 1))) * 100
    
    # Additional metrics
    mean_actual = np.mean(actual_clean)
    mean_predicted = np.mean(predicted_clean)
    bias = np.mean(predicted_clean - actual_clean)
    bias_pct = (bias / mean_actual) * 100 if mean_actual != 0 else 0
    
    # R-squared
    ss_res = np.sum((actual_clean - predicted_clean) ** 2)
    ss_tot = np.sum((actual_clean - mean_actual) ** 2)
    r2 = 1 - (ss_res / ss_tot) if ss_tot != 0 else 0
    
    return {
        'Method': method_name,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape,
        'R2': r2,
        'Bias': bias,
        'Bias_Percent': bias_pct,
        'Mean_Actual': mean_actual,
        'Mean_Predicted': mean_predicted,
        'Data_Points': len(actual_clean)
    }

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 170, Finished, Available, Finished)

In [169]:
def forecast_overall(data, exog_vars, forecast_steps, forecast_dates):
    """
    Generate overall forecasts using multiple methods and ensemble approach
    
    Parameters:
    - data: DataFrame with overall time series data
    - exog_vars: List of exogenous variables to use in forecasting
    - forecast_steps: Number of steps to forecast
    - forecast_dates: Date range for forecasts
    
    Returns:
    - DataFrame with all forecast methods and ensemble result
    """
    print("=== LEVEL 0: OVERALL FORECASTING ===")
    
    # Prepare overall data
    overall_series = data.set_index('Date')['Quantity Invoiced']
    display(data.head())
    overall_exog = data.set_index('Date')[exog_vars]
    
    # Generate forecasts using different methods
    print("Generating ARIMA forecast...")
    overall_arima_forecast, overall_arima_conf, overall_arima_aic = forecast_arima(overall_series, forecast_steps)
    
    print("Generating SARIMA forecast...")
    overall_sarima_forecast, overall_sarima_conf, overall_sarima_aic = forecast_sarima(overall_series, forecast_steps, overall_exog)
    
    print("Generating Exponential Smoothing forecast...")
    overall_exp_forecast, overall_exp_conf, overall_exp_aic = forecast_exponential_smoothing(overall_series, forecast_steps)
    
    print("Generating Prophet forecast...")
    overall_prophet_forecast, overall_prophet_conf, overall_prophet_aic = forecast_prophet(overall_series, forecast_steps, overall_exog)

    
    # Print exponential smoothing forecast
    print("Exponential Smoothing Forecast:")
    print(overall_exp_forecast.head())

    # Create ensemble forecast
    aics = [overall_arima_aic, overall_sarima_aic, overall_exp_aic, overall_prophet_aic]    
    overall_ensemble_forecast = ensemble_forecast(
        [overall_arima_forecast, overall_sarima_forecast, overall_exp_forecast, overall_prophet_forecast], 
        aics
    )

    print("Forecast Method Value Lengths:")
    print(f"  ARIMA: {len(overall_arima_forecast)}, SARIMA: {len(overall_sarima_forecast)}, "
        f"ExpSmoothing: {len(overall_exp_forecast)}, Prophet: {len(overall_prophet_forecast)}, "
        f"Ensemble: {len(overall_ensemble_forecast)}")

    
    # Store overall forecasts
    overall_forecasts = pd.DataFrame({
        'Date': forecast_dates,
        'ARIMA': overall_arima_forecast.values,
        'SARIMA': overall_sarima_forecast.values,
        'ExpSmoothing': overall_exp_forecast.values,
        'Prophet': overall_prophet_forecast.values,
        'Ensemble': overall_ensemble_forecast.values,
        'Level': 'Overall',
        'Segment': 'Total'
    })
    
    print(f"Overall forecast range: {overall_ensemble_forecast.min():,.0f} - {overall_ensemble_forecast.max():,.0f}")
    
    return overall_forecasts

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 171, Finished, Available, Finished)

# Running Overall Quantity Forecasting

Execute the overall quantity forecasting using multiple methods and ensemble approach.

In [170]:
# Set forecasting parameters
FORECAST_STEPS = 12  # 12 months ahead
START_DATE = data['Date'].max()
FORECAST_DATES = pd.date_range(start=START_DATE, periods=FORECAST_STEPS, freq='MS')

# Generate overall forecasts
overall_forecasts = forecast_overall(
    data=data,
    exog_vars=exog_vars,
    forecast_steps=FORECAST_STEPS,
    forecast_dates=FORECAST_DATES
)

overall_forecasts

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 172, Finished, Available, Finished)

=== LEVEL 0: OVERALL FORECASTING ===


SynapseWidget(Synapse.DataFrame, 2a6efcb7-4591-467f-9f80-8bee094bbcfb)

Generating ARIMA forecast...


Generating SARIMA forecast...


Generating Exponential Smoothing forecast...


Generating Prophet forecast...
Exponential Smoothing Forecast:
2025-02-01    2.506060e+07
2025-03-01    2.741327e+07
2025-04-01    2.338350e+07
2025-05-01    2.386932e+07
2025-06-01    2.577085e+07
Freq: MS, dtype: float64
Model weights - ARIMA: 0.056, SARIMA: 0.062, EXP: 0.061, Prophet: 0.821
Forecast Method Value Lengths:
  ARIMA: 12, SARIMA: 12, ExpSmoothing: 12, Prophet: 12, Ensemble: 12
Overall forecast range: 15,984,964 - 26,696,505


,Date,ARIMA,SARIMA,ExpSmoothing,Prophet,Ensemble,Level,Segment
0,2025-01-01,2.395974e+07,2.466102e+07,2.506060e+07,2.369124e+07,2.385040e+07,Overall,Total
1,2025-02-01,2.377392e+07,2.733635e+07,2.741327e+07,2.647905e+07,2.643931e+07,Overall,Total
2,2025-03-01,2.375142e+07,2.379334e+07,2.338350e+07,2.221973e+07,2.247381e+07,Overall,Total
3,2025-04-01,2.374870e+07,2.376519e+07,2.386932e+07,2.260067e+07,2.281455e+07,Overall,Total
4,2025-05-01,2.374837e+07,2.385763e+07,2.577085e+07,2.590983e+07,2.565413e+07,Overall,Total
5,2025-06-01,2.374833e+07,2.692809e+07,2.702657e+07,2.608888e+07,2.606849e+07,Overall,Total
6,2025-07-01,2.374832e+07,2.533405e+07,2.525842e+07,2.441556e+07,2.448720e+07,Overall,Total
7,2025-08-01,2.374832e+07,2.585251e+07,2.641192e+07,2.698093e+07,2.669651e+07,Overall,Total
8,2025-09-01,2.374832e+07,2.408483e+07,2.407482e+07,2.336418e+07,2.347384e+07,Overall,Total
9,2025-10-01,2.374832e+07,2.278209e+07,2.347627e+07,2.282043e+07,2.290992e+07,Overall,Total


## Overall Forecast Visualization & Results

Let's visualize the overall forecasts and compare them with historical data.

In [171]:
# Create comprehensive forecast visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Overall Forecast: Historical vs Predicted', 'Forecast Methods Comparison', 
                    'Forecast Trend Analysis', 'Forecast Method Performance'),
    vertical_spacing=0.12,
    horizontal_spacing=0.10,
    specs=[[{"colspan": 2}, None],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Overall forecast plot (spans both columns in first row)
fig.add_trace(
    go.Scatter(x=data['Date'], y=data['Quantity Invoiced'],
              mode='lines+markers', name='Historical Total',
              line=dict(color='darkblue', width=2),
              marker=dict(size=4)),
    row=1, col=1
)

# Add forecast methods
forecast_colors = {'ARIMA': 'red', 'SARIMA': 'green', 'ExpSmoothing': 'orange', 'Prophet': 'purple', 'Ensemble': 'black'}
for method, color in forecast_colors.items():
    if method in overall_forecasts.columns:
        fig.add_trace(
            go.Scatter(x=overall_forecasts['Date'], y=overall_forecasts[method],
                      mode='lines+markers', name=f'{method} Forecast',
                      line=dict(color=color, width=3 if method == 'Ensemble' else 2, 
                               dash='solid' if method == 'Ensemble' else 'dash'),
                      marker=dict(size=6 if method == 'Ensemble' else 4)),
            row=1, col=1
        )

# Method comparison - just the forecast values
forecast_methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']
method_colors = ['red', 'green', 'orange', 'purple', 'black']

for method, color in zip(forecast_methods, method_colors):
    if method in overall_forecasts.columns:
        fig.add_trace(
            go.Scatter(x=overall_forecasts['Date'], y=overall_forecasts[method],
                      mode='lines+markers', name=f'{method}',
                      line=dict(color=color, width=2),
                      marker=dict(size=5),
                      showlegend=False),
            row=2, col=1
        )

# Forecast statistics by method
if all(method in overall_forecasts.columns for method in forecast_methods):
    method_stats = []
    for method in forecast_methods:
        mean_val = overall_forecasts[method].mean()
        std_val = overall_forecasts[method].std()
        method_stats.append({'Method': method, 'Mean': mean_val, 'Std': std_val})
    
    stats_df = pd.DataFrame(method_stats)
    
    fig.add_trace(
        go.Bar(x=stats_df['Method'], y=stats_df['Mean'],
               name='Mean Forecast',
               marker_color=['red', 'green', 'orange', 'purple', 'black'],
               showlegend=False),
        row=2, col=2
    )

fig.update_layout(
    height=800,
    title_text="Overall Sales Quantity Forecasting: Historical vs Predicted (12-Month Forecast)",
    showlegend=True,
    legend=dict(orientation="v", yanchor="top", y=1, xanchor="left", x=1.02),
    template="plotly_white"
)

fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Method", row=2, col=2)

fig.update_yaxes(title_text="Total Quantity", row=1, col=1)
fig.update_yaxes(title_text="Total Quantity", row=2, col=1)
fig.update_yaxes(title_text="Mean Forecast", row=2, col=2)

fig.show()

# Print forecast summary
print("\n=== OVERALL FORECAST SUMMARY ===")
print(f"Forecast Period: {FORECAST_DATES[0].strftime('%Y-%m')} to {FORECAST_DATES[-1].strftime('%Y-%m')}")
print(f"\nOverall Forecast Summary:")
print(f"  Mean Monthly Forecast: {overall_forecasts['Ensemble'].mean():,.0f}")
print(f"  Total 12-Month Forecast: {overall_forecasts['Ensemble'].sum():,.0f}")
print(f"  Min-Max Range: {overall_forecasts['Ensemble'].min():,.0f} - {overall_forecasts['Ensemble'].max():,.0f}")

# Historical comparison
historical_mean = data['Quantity Invoiced'].mean()
historical_total_last_12 = data['Quantity Invoiced'].tail(12).sum()

print(f"\nHistorical Comparison:")
print(f"  Historical Mean Monthly: {historical_mean:,.0f}")
print(f"  Historical Last 12-Month Total: {historical_total_last_12:,.0f}")
print(f"  Forecast vs Historical Mean: {((overall_forecasts['Ensemble'].mean() / historical_mean - 1) * 100):+.1f}%")
print(f"  Forecast vs Last 12-Month Total: {((overall_forecasts['Ensemble'].sum() / historical_total_last_12 - 1) * 100):+.1f}%")

# Method comparison
print(f"\nMethod Comparison (12-Month Totals):")
for method in forecast_methods:
    if method in overall_forecasts.columns:
        total_forecast = overall_forecasts[method].sum()
        vs_ensemble = ((total_forecast / overall_forecasts['Ensemble'].sum() - 1) * 100)
        print(f"  {method}: {total_forecast:,.0f} ({vs_ensemble:+.1f}% vs Ensemble)")

# Seasonal analysis of forecast
# print(f"\nSeasonal Forecast Analysis:")
# overall_forecasts['Month'] = overall_forecasts['Date'].dt.month_name()
# monthly_forecast = overall_forecasts.groupby('Month')['Ensemble'].mean()
# print("  Average Monthly Forecast:")
# for month, value in monthly_forecast.items():
#     print(f"    {month}: {value:,.0f}")

# # Show key forecast periods
# print(f"\nKey Forecast Periods:")
# max_month = overall_forecasts.loc[overall_forecasts['Ensemble'].idxmax()]
# min_month = overall_forecasts.loc[overall_forecasts['Ensemble'].idxmin()]
# print(f"  Highest Forecast: {max_month['Date'].strftime('%Y-%m')} ({max_month['Ensemble']:,.0f})")
# print(f"  Lowest Forecast: {min_month['Date'].strftime('%Y-%m')} ({min_month['Ensemble']:,.0f})")

# Growth trajectory
forecast_growth = ((overall_forecasts['Ensemble'].iloc[-1] / overall_forecasts['Ensemble'].iloc[0] - 1) * 100)
print(f"  Forecast Growth (First to Last Month): {forecast_growth:+.1f}%")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 173, Finished, Available, Finished)


=== OVERALL FORECAST SUMMARY ===
Forecast Period: 2025-01 to 2025-12

Overall Forecast Summary:
  Mean Monthly Forecast: 23,820,090
  Total 12-Month Forecast: 285,841,079
  Min-Max Range: 15,984,964 - 26,696,505

Historical Comparison:
  Historical Mean Monthly: 25,694,142
  Historical Last 12-Month Total: 292,848,063
  Forecast vs Historical Mean: -7.3%
  Forecast vs Last 12-Month Total: -2.4%

Method Comparison (12-Month Totals):
  ARIMA: 285,220,401 (-0.2% vs Ensemble)
  SARIMA: 289,967,444 (+1.4% vs Ensemble)
  ExpSmoothing: 294,029,950 (+2.9% vs Ensemble)
  Prophet: 284,958,666 (-0.3% vs Ensemble)
  Ensemble: 285,841,079 (+0.0% vs Ensemble)
  Forecast Growth (First to Last Month): +4.8%


## Export Results

Let's save the overall forecasts to CSV files for further analysis and reporting.

In [172]:
# Export forecast results
print("=== EXPORTING OVERALL FORECAST RESULTS ===")

# Overall forecast export
overall_export = overall_forecasts[['Date', 'ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']].copy()
overall_export.to_csv(data_location + 'modelGeneratedData/overall_sales_forecast.csv', index=False)
print(f"Overall forecasts exported to: overall_sales_forecast.csv")

# Summary report
summary_report = []
summary_report.append({
    'Level': 'Overall',
    'Segment': 'Total',
    'Total_12Month_Forecast': overall_forecasts['Ensemble'].sum(),
    'Average_Monthly_Forecast': overall_forecasts['Ensemble'].mean(),
    'Min_Monthly_Forecast': overall_forecasts['Ensemble'].min(),
    'Max_Monthly_Forecast': overall_forecasts['Ensemble'].max(),
    'Forecast_Growth_Rate': ((overall_forecasts['Ensemble'].iloc[-1] / overall_forecasts['Ensemble'].iloc[0] - 1) * 100),
    'Vs_Historical_Mean': ((overall_forecasts['Ensemble'].mean() / data['Quantity Invoiced'].mean() - 1) * 100)
})

# Add method-specific summaries
for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet']:
    if method in overall_forecasts.columns:
        summary_report.append({
            'Level': 'Overall',
            'Segment': f'Total_{method}',
            'Total_12Month_Forecast': overall_forecasts[method].sum(),
            'Average_Monthly_Forecast': overall_forecasts[method].mean(),
            'Min_Monthly_Forecast': overall_forecasts[method].min(),
            'Max_Monthly_Forecast': overall_forecasts[method].max(),
            'Forecast_Growth_Rate': ((overall_forecasts[method].iloc[-1] / overall_forecasts[method].iloc[0] - 1) * 100),
            'Vs_Historical_Mean': ((overall_forecasts[method].mean() / data['Quantity Invoiced'].mean() - 1) * 100)
        })

summary_df = pd.DataFrame(summary_report)
summary_df.to_csv(data_location + 'modelGeneratedData/forecast_metrics_summary.csv', index=False)
print(f"Forecast summary report exported to: forecast_metrics_summary.csv")

# Create detailed monthly forecast breakdown
monthly_breakdown = overall_forecasts.copy()
monthly_breakdown['Year'] = monthly_breakdown['Date'].dt.year
monthly_breakdown['Month'] = monthly_breakdown['Date'].dt.month
monthly_breakdown['Month_Name'] = monthly_breakdown['Date'].dt.month_name()
monthly_breakdown['Quarter'] = monthly_breakdown['Date'].dt.quarter

# Calculate quarterly summaries
quarterly_summary = monthly_breakdown.groupby(['Year', 'Quarter']).agg({
    'ARIMA': 'sum',
    'SARIMA': 'sum', 
    'ExpSmoothing': 'sum',
    'Prophet': 'sum',
    'Ensemble': 'sum'
}).reset_index()
quarterly_summary['Quarter_Label'] = quarterly_summary['Year'].astype(str) + '-Q' + quarterly_summary['Quarter'].astype(str)

quarterly_summary.to_csv(data_location + 'modelGeneratedData/quarterly_forecast_summary.csv', index=False)
print(f"Quarterly forecast summary exported to: quarterly_forecast_summary.csv")

print(f"\n=== EXPORT COMPLETE ===")
print(f"Total files exported: 3")
print(f"Export timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Display final summary
print(f"\n=== FINAL FORECAST SUMMARY ===")
display(summary_df)

print(f"\n=== QUARTERLY FORECAST BREAKDOWN ===")
display(quarterly_summary[['Quarter_Label', 'ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']])

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 174, Finished, Available, Finished)

=== EXPORTING OVERALL FORECAST RESULTS ===
Overall forecasts exported to: overall_sales_forecast.csv
Forecast summary report exported to: forecast_metrics_summary.csv
Quarterly forecast summary exported to: quarterly_forecast_summary.csv

=== EXPORT COMPLETE ===
Total files exported: 3
Export timestamp: 2025-07-27 00:05:25

=== FINAL FORECAST SUMMARY ===


SynapseWidget(Synapse.DataFrame, 7e312d4e-a5ec-46eb-ab44-468c8f2dcaf8)


=== QUARTERLY FORECAST BREAKDOWN ===


SynapseWidget(Synapse.DataFrame, 01d0b983-54ce-4b80-a564-d0ab5f73533f)

## 2025 Actuals Comparison

In [173]:
# Compute the total Quantity Invoiced for each month of 2025
# 2025 Actuals Comparison so far
print("\n=== 2025 ACTUALS COMPARISON ===")

raw_df = pd.read_csv(data_location + "userProvidedData/2015-2025 Qty.csv")
print(raw_df.columns)

# Loading Quantity data in pandas DataFrames
# 2015 to 2025 Qty.csv
actual_quantity_df = pd.read_csv(data_location + "userProvidedData/2015-2025 Qty.csv", parse_dates=['Fiscal_Hierarchy_-_Full_Date'])
actual_quantity_df = actual_quantity_df[actual_quantity_df['Fiscal_Hierarchy_-_Full_Date'].dt.year == 2025]

# Ensure Quantity Invoiced is numeric
actual_quantity_df['Quantity_Invoiced'] = actual_quantity_df['Quantity_Invoiced'].astype(str).str.replace(',', '')
actual_quantity_df['Quantity_Invoiced'] = pd.to_numeric(actual_quantity_df['Quantity_Invoiced'], errors='coerce')

# # Filtering Later Quantity Data to match
# print(f"Before filtering quantity data shape: {actual_quantity_df.shape}")

# # Filter to InterCompany
# actual_quantity_df = actual_quantity_df[actual_quantity_df['CustomerSegment'] == 'InterCompany']
# print(f"After filtering to InterCompany segment, quantity data shape: {actual_quantity_df.shape}")

# Group by month and sum the Quantity Invoiced
actual_quantity_df = actual_quantity_df.groupby(actual_quantity_df['Fiscal_Hierarchy_-_Full_Date'].dt.to_period('M'))['Quantity_Invoiced'].sum().reset_index()
actual_quantity_df['Fiscal_Hierarchy_-_Full_Date'] = actual_quantity_df['Fiscal_Hierarchy_-_Full_Date'].dt.to_timestamp()
# Rename columns for clarity
actual_quantity_df.rename(columns={'Fiscal_Hierarchy_-_Full_Date': 'Date', 'Quantity_Invoiced': 'Actual_Quantity'}, inplace=True)

# Display the monthly actuals for 2025
display(actual_quantity_df.head())


StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 175, Finished, Available, Finished)


=== 2025 ACTUALS COMPARISON ===
Index(['ParentAccountName', 'CustomerSegment', 'Customer Number',
       'CategoryName', 'SubCategoryName', 'RiekeUniversalProductCode',
       'Quantity_Invoiced', 'Fiscal_Hierarchy_-_Fiscal_Year',
       'Fiscal Hierarchy - Fiscal Semester',
       'Fiscal Hierarchy - Fiscal Quarter', 'Fiscal_Hierarchy_-_Fiscal_Month',
       'Fiscal_Hierarchy_-_Full_Date', 'TriMasEndMarketsLevel_1',
       'TriMasEndMarketsLevel_2'],
      dtype='object')


SynapseWidget(Synapse.DataFrame, f97082ef-f04f-4d5b-a8f3-dccf994cf34e)

In [174]:
# Create a CSV file with 2025 actuals and forecast comparison by combining with overall_export
comparison_df = overall_forecasts.merge(actual_quantity_df, on='Date', how='left')
# Drop the 'Level' and 'Segment' columns from overall_export
comparison_df = comparison_df.drop(columns=['Level', 'Segment'])
# Only use rows where Actual_Quantity is not NaN
comparison_df = comparison_df[~comparison_df['Actual_Quantity'].isna()]
# Export as CSV
comparison_df.to_csv(data_location + 'modelGeneratedData/2025_actuals_forecast_comparison.csv', index=False)
print(f"2025 actuals vs forecast comparison exported to: 2025_actuals_forecast_comparison.csv")

forecast_methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet']

# Display comparison df
display(comparison_df)

# Calculate accuracy metrics for each method
accuracy_results = []
for method in forecast_methods:
    if method in comparison_df.columns:
        metrics = calculate_accuracy_metrics(
            comparison_df['Actual_Quantity'].values,
            comparison_df[method].values,
            method
        )
        if metrics:
            accuracy_results.append(metrics)

# Convert results to DataFrame and display
accuracy_df = pd.DataFrame(accuracy_results)
display(accuracy_df)

# Export accuracy metrics to CSV
accuracy_df.to_csv(data_location + 'modelGeneratedData/2025_forecast_accuracy_metrics.csv', index=False)

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 176, Finished, Available, Finished)

2025 actuals vs forecast comparison exported to: 2025_actuals_forecast_comparison.csv


SynapseWidget(Synapse.DataFrame, d0245af5-3875-4dff-b4da-8f1f3952274a)

SynapseWidget(Synapse.DataFrame, cd18d1bd-fa7b-459d-ad57-c709efc72621)

In [175]:
# Plot forcasts methods vs actuals
fig = go.Figure()
# Plot Actual Quantity
fig.add_trace(
    go.Scatter(x=comparison_df['Date'], y=comparison_df['Actual_Quantity'],
              mode='lines+markers', name='Actual Quantity',
              line=dict(color='black', width=2),
              marker=dict(size=4))
)

# Plot each forecast method as a dotted line
for method in forecast_methods:
    if method in comparison_df.columns:
        fig.add_trace(
            go.Scatter(
                x=comparison_df['Date'],
                y=comparison_df[method],
                mode='lines+markers',
                name=f'{method} Forecast',
                line=dict(width=2, dash='dot'),
                marker=dict(size=4)
            )
        )

# Show the plot
fig.show()

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 177, Finished, Available, Finished)

# Backtesting Analysis

Let's evaluate the performance of our forecasting models by backtesting on previous data. For 2024, we'll train models on data up to 2023 and test predictions against actual 2024 values.

In [176]:
# Backtesting Setup: Split data into train (2015-2023) and test (2024)
print("=== BACKTESTING SETUP ===")

# Define split date
BACKTEST_SPLIT_DATE = pd.to_datetime('2024-01-01') #year 2024-01-01 etc.
# Forecast horizon
FORECAST_HORIZON = 12
print(f"Training data: Before {BACKTEST_SPLIT_DATE}")
print(f"Test data: {BACKTEST_SPLIT_DATE} onwards")

# Check if we have test data
data_test = data[(data['Date'] >= BACKTEST_SPLIT_DATE) & (data['Date'] < BACKTEST_SPLIT_DATE + pd.DateOffset(months=FORECAST_HORIZON))]

# Setting the data train: 2015-01 to 2022-12 (exclude 2023)
# data_train = data[data['Date'] < BACKTEST_SPLIT_DATE] # Original
data_train = data[(data['Date'] >= '2015-01-01') & (data['Date'] < '2023-01-01')]
 

# Display training and test data columns
print(f"Training data columns: {data_train.columns.tolist()}")
print(f"Test data columns: {data_test.columns.tolist()}")

print(f"Total data points: {len(data)}")
print(f"Training data points: {len(data_train)}")
print(f"Test data points: {len(data_test)}")
print(f"Test Data range: {data_test['Date'].min()} to {data_test['Date'].max()}")

if len(data_test) == 0:
    print("⚠️ No data available for backtesting!")
else:
    print(f"✅ Found {len(data_test)} data points for backtesting")
    print(f"Test months available: {sorted(data_test['Date'].dt.strftime('%Y-%m').unique())}")

# Create training overall aggregation
print("\n=== CREATING TRAINING AGGREGATION ===")

# Overall training data
print(f"Training data shape: {data_train.shape}")
print(f"Training period: {data_train['Date'].min()} to {data_train['Date'].max()}")

# Create actual aggregations for comparison
print("\n=== CREATING ACTUAL AGGREGATIONS ===")

if len(data_test) > 0:
    # Overall actuals
    overall_actual = data_test.groupby('Date').agg({
        'Quantity Invoiced': 'sum',
    }).reset_index()
    
    print(f"Actual data shape: {overall_actual.shape}")
    print(f"Actual period: {overall_actual['Date'].min()} to {overall_actual['Date'].max()}")
    
    print(f"\nOverall monthly totals:")
    for _, row in overall_actual.iterrows():
        print(f"  {row['Date'].strftime('%Y-%m')}: {row['Quantity Invoiced']:,.0f}")
        
    # Basic statistics
    print(f"\nBacktesting Statistics:")
    print(f"  Training samples: {len(data_train)}")
    print(f"  Test samples: {len(overall_actual)}")
    print(f"  Training mean: {data_train['Quantity Invoiced'].mean():,.0f}")
    print(f"  Test mean: {overall_actual['Quantity Invoiced'].mean():,.0f}")
    print(f"  Training std: {data_train['Quantity Invoiced'].std():,.0f}")
    print(f"  Test std: {overall_actual['Quantity Invoiced'].std():,.0f}")
else:
    print("No data available for comparison")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 178, Finished, Available, Finished)

=== BACKTESTING SETUP ===
Training data: Before 2024-01-01 00:00:00
Test data: 2024-01-01 00:00:00 onwards
Training data columns: ['Date', 'Quantity Invoiced', 'Quantity Invoiced Mean', 'Quantity Invoiced Count', 'Unique Customers', 'Unique Products', 'Unique Categories', 'Unique SubCategories', 'Unique EndMarkets L1', 'Unique EndMarkets L2', 'future_orders_count', 'future_orders_qty_total', 'future_orders_qty_next_1m', 'future_orders_qty_next_3m', 'future_orders_qty_next_6m', 'future_orders_qty_next_12m', 'future_orders_avg_lead_time', 'future_orders_min_lead_time', 'future_orders_max_lead_time', 'future_orders_due_next_month', 'future_orders_due_next_quarter', 'future_orders_unique_customers', 'future_orders_unique_products', 'future_to_current_ratio', 'qty_rolling_avg_3m', 'future_to_rolling_3m_ratio', 'qty_rolling_avg_12m', 'future_to_rolling_12m_ratio', 'has_future_orders', 'high_future_orders', 'future_customer_diversity_ratio', 'future_product_diversity_ratio', 'PP_Spot', 'Resin

In [177]:
# Generate Backtesting Forecasts (Train on 2015-2023, Predict 2024)
if len(data_test) > 0:
    print("=== OVERALL FORECASTING BACKTESTING ===")
    
    # Determine how many months of data we have
    backtest_months = len(overall_actual)
    backtest_dates = pd.date_range(start=BACKTEST_SPLIT_DATE, periods=backtest_months, freq='MS')
    
    print(f"Generating overall forecasts for {backtest_months} months")
    
    # Generate Overall Backtest Forecast using the reusable function
    print("\nGenerating Overall Backtest Forecast...")
    overall_backtest_forecasts = forecast_overall(
        data=data_train,
        exog_vars=exog_vars,
        forecast_steps=backtest_months,
        forecast_dates=backtest_dates
    )
    
    # Store backtest forecasts for overall level
    backtest_forecasts = overall_backtest_forecasts.copy()
    
    # Add actual values
    backtest_forecasts = backtest_forecasts.merge(
        overall_actual[['Date', 'Quantity Invoiced']], 
        on='Date', 
        how='left'
    )
    backtest_forecasts.rename(columns={'Quantity Invoiced': 'Actual'}, inplace=True)
    
    print(f"Backtest forecast range:")
    print(f"  ARIMA: {backtest_forecasts['ARIMA'].min():,.0f} - {backtest_forecasts['ARIMA'].max():,.0f}")
    print(f"  SARIMA: {backtest_forecasts['SARIMA'].min():,.0f} - {backtest_forecasts['SARIMA'].max():,.0f}")
    print(f"  ExpSmoothing: {backtest_forecasts['ExpSmoothing'].min():,.0f} - {backtest_forecasts['ExpSmoothing'].max():,.0f}")
    print(f"  Prophet: {backtest_forecasts['Prophet'].min():,.0f} - {backtest_forecasts['Prophet'].max():,.0f}")
    print(f"  Ensemble: {backtest_forecasts['Ensemble'].min():,.0f} - {backtest_forecasts['Ensemble'].max():,.0f}")
    print(f"Actual range: {overall_actual['Quantity Invoiced'].min():,.0f} - {overall_actual['Quantity Invoiced'].max():,.0f}")
    
    print("\nBacktest vs Actual comparison:")
    display(backtest_forecasts)
    
    # Calculate forecast errors for quick assessment
    if 'Actual' in backtest_forecasts.columns:
        for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']:
            if method in backtest_forecasts.columns:
                mae = np.mean(np.abs(backtest_forecasts['Actual'] - backtest_forecasts[method]))
                mape = np.mean(np.abs((backtest_forecasts['Actual'] - backtest_forecasts[method]) / backtest_forecasts['Actual'])) * 100
                print(f"  {method} - MAE: {mae:,.0f}, MAPE: {mape:.2f}%")
    
else:
    print("⚠️ Skipping backtesting - no 2024 data available")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 179, Finished, Available, Finished)

=== OVERALL FORECASTING BACKTESTING ===
Generating overall forecasts for 12 months

Generating Overall Backtest Forecast...
=== LEVEL 0: OVERALL FORECASTING ===


SynapseWidget(Synapse.DataFrame, bd024ce3-5c2e-4e56-9d4a-b7e64705b211)

Generating ARIMA forecast...


Generating SARIMA forecast...


Generating Exponential Smoothing forecast...


Generating Prophet forecast...
Exponential Smoothing Forecast:
2023-01-01    2.575506e+07
2023-02-01    2.453187e+07
2023-03-01    2.762870e+07
2023-04-01    2.340056e+07
2023-05-01    2.424134e+07
Freq: MS, dtype: float64
Model weights - ARIMA: 0.057, SARIMA: 0.065, EXP: 0.063, Prophet: 0.816
Forecast Method Value Lengths:
  ARIMA: 12, SARIMA: 12, ExpSmoothing: 12, Prophet: 12, Ensemble: 12
Overall forecast range: 17,833,833 - 25,868,423
Backtest forecast range:
  ARIMA: 24,119,452 - 24,831,433
  SARIMA: 6,370,908 - 23,119,284
  ExpSmoothing: 16,919,733 - 27,628,704
  Prophet: 18,327,627 - 26,055,846
  Ensemble: 17,833,833 - 25,868,423
Actual range: 15,512,968 - 28,726,653

Backtest vs Actual comparison:


SynapseWidget(Synapse.DataFrame, bea88386-1084-4591-9940-8595054fb562)

  ARIMA - MAE: 2,607,669, MAPE: 12.53%
  SARIMA - MAE: 9,225,875, MAPE: 37.48%
  ExpSmoothing - MAE: 2,010,660, MAPE: 8.81%
  Prophet - MAE: 2,418,901, MAPE: 10.35%
  Ensemble - MAE: 2,663,239, MAPE: 11.11%


In [178]:
# Calculate Backtesting Accuracy Metrics
if len(data_test) > 0:
    print("\n=== BACKTESTING ACCURACY METRICS ===")
    
    # Calculate metrics for each method
    accuracy_results = []
    
    # Check which methods are available in the forecast data
    methods = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']
    method_names = ['ARIMA', 'SARIMA', 'Exponential Smoothing', 'Prophet', 'Ensemble']
    
    for method, name in zip(methods, method_names):
        if method in backtest_forecasts.columns:
            metrics = calculate_accuracy_metrics(
                backtest_forecasts['Actual'].values,
                backtest_forecasts[method].values,
                name
            )
            if metrics:
                accuracy_results.append(metrics)
    
    # Convert to DataFrame
    accuracy_df = pd.DataFrame(accuracy_results)
    
    print("Overall Backtesting Accuracy Results:")
    print("="*80)
    for _, row in accuracy_df.iterrows():
        print(f"\n{row['Method']} Performance:")
        print(f"  MAE (Mean Absolute Error): {row['MAE']:,.0f}")
        print(f"  RMSE (Root Mean Square Error): {row['RMSE']:,.0f}")
        print(f"  MAPE (Mean Absolute Percentage Error): {row['MAPE']:.2f}%")
        print(f"  R² (Coefficient of Determination): {row['R2']:.4f}")
        print(f"  Bias: {row['Bias']:,.0f} ({row['Bias_Percent']:.2f}%)")
        print(f"  Mean Actual: {row['Mean_Actual']:,.0f}")
        print(f"  Mean Predicted: {row['Mean_Predicted']:,.0f}")
    
    # Find best performing method
    best_method = accuracy_df.loc[accuracy_df['MAPE'].idxmin(), 'Method']
    best_mape = accuracy_df.loc[accuracy_df['MAPE'].idxmin(), 'MAPE']
    
    print(f"\n🏆 Best Performing Method: {best_method} (MAPE: {best_mape:.2f}%)")
    
    # Export accuracy results
    accuracy_df.to_csv(data_location + 'modelGeneratedData/backtesting_accuracy_metrics.csv', index=False)
    print(f"\n📊 Accuracy metrics exported to: backtesting_accuracy_metrics.csv")
    
    display(accuracy_df)
else:
    print("⚠️ Skipping accuracy calculation - no 2024 data available")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 180, Finished, Available, Finished)


=== BACKTESTING ACCURACY METRICS ===
Overall Backtesting Accuracy Results:

ARIMA Performance:
  MAE (Mean Absolute Error): 2,607,669
  RMSE (Root Mean Square Error): 3,577,118
  MAPE (Mean Absolute Percentage Error): 12.53%
  R² (Coefficient of Determination): -0.0265
  Bias: 343,471 (1.41%)
  Mean Actual: 24,418,603
  Mean Predicted: 24,762,074

SARIMA Performance:
  MAE (Mean Absolute Error): 9,225,875
  RMSE (Root Mean Square Error): 11,059,527
  MAPE (Mean Absolute Percentage Error): 37.48%
  R² (Coefficient of Determination): -8.8123
  Bias: -9,225,875 (-37.78%)
  Mean Actual: 24,418,603
  Mean Predicted: 15,192,728

Exponential Smoothing Performance:
  MAE (Mean Absolute Error): 2,010,660
  RMSE (Root Mean Square Error): 2,617,345
  MAPE (Mean Absolute Percentage Error): 8.81%
  R² (Coefficient of Determination): 0.4504
  Bias: 94,803 (0.39%)
  Mean Actual: 24,418,603
  Mean Predicted: 24,513,406

Prophet Performance:
  MAE (Mean Absolute Error): 2,418,901
  RMSE (Root Mean Squ

SynapseWidget(Synapse.DataFrame, a87b51c4-f568-4f11-bae6-a9fba76fefd0)

### Backtesting Visualization

In [179]:
# Backtesting Visualization
if len(data_test) > 0:
    print("\n=== BACKTESTING VISUALIZATION ===")
    
    # Create comprehensive backtesting visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Forecast vs Actual (Overall)', 
            'Forecast Accuracy by Method (MAPE)',
            'Residuals Analysis (Ensemble)',
            'Method Performance Comparison'
        ),
        specs=[[{"secondary_y": False}, {"type": "bar"}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # Plot 1: Forecast vs Actual
    fig.add_trace(
        go.Scatter(
            x=backtest_forecasts['Date'], 
            y=backtest_forecasts['Actual'],
            mode='lines+markers', 
            name='Actual',
            line=dict(color='darkblue', width=3),
            marker=dict(size=6)
        ), row=1, col=1
    )
    
    # Add forecast methods
    colors_bt = ['red', 'green', 'orange', 'purple', 'black']
    methods_bt = ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']
    method_labels = ['ARIMA', 'SARIMA', 'Exp Smoothing', 'Prophet', 'Ensemble']
    
    for method, label, color in zip(methods_bt, method_labels, colors_bt):
        if method in backtest_forecasts.columns:
            fig.add_trace(
                go.Scatter(
                    x=backtest_forecasts['Date'], 
                    y=backtest_forecasts[method],
                    mode='lines+markers', 
                    name=f'{label} Forecast',
                    line=dict(color=color, width=3 if method == 'Ensemble' else 2, 
                             dash='solid' if method == 'Ensemble' else 'dash'),
                    marker=dict(size=6 if method == 'Ensemble' else 4)
                ), row=1, col=1
            )
    
    # Plot 2: MAPE Comparison
    if len(accuracy_results) > 0:
        fig.add_trace(
            go.Bar(
                x=[r['Method'] for r in accuracy_results],
                y=[r['MAPE'] for r in accuracy_results],
                name='MAPE %',
                marker_color=['red' if r['Method'] == best_method else 'lightblue' for r in accuracy_results],
                showlegend=False
            ), row=1, col=2
        )
    
    # Plot 3: Residuals (Actual - Ensemble Forecast)
    if 'Ensemble' in backtest_forecasts.columns:
        residuals = backtest_forecasts['Actual'] - backtest_forecasts['Ensemble']
        fig.add_trace(
            go.Scatter(
                x=backtest_forecasts['Date'], 
                y=residuals,
                mode='lines+markers', 
                name='Residuals (Actual - Ensemble)',
                line=dict(color='red', width=2),
                marker=dict(size=5),
                showlegend=False
            ), row=2, col=1
        )
        
        # Add zero line
        fig.add_hline(y=0, line_dash="dash", line_color="black", row=2, col=1)
    
    # Plot 4: Method Performance Metrics
    if len(accuracy_results) > 0:
        methods = [r['Method'] for r in accuracy_results]
        mae_values = [r['MAE'] for r in accuracy_results]
        
        # Create a normalized view of MAE for comparison
        fig.add_trace(
            go.Bar(
                x=methods,
                y=mae_values,
                name='MAE',
                marker_color=['gold' if m == best_method else 'lightcoral' for m in methods],
                showlegend=False
            ), row=2, col=2
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        title_text="Overall Forecasting Backtesting Analysis: Performance & Accuracy Assessment",
        showlegend=True,
        template="plotly_white"
    )
    
    # Update axes labels
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Method", row=1, col=2)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_xaxes(title_text="Method", row=2, col=2)
    
    fig.update_yaxes(title_text="Quantity", row=1, col=1)
    fig.update_yaxes(title_text="MAPE (%)", row=1, col=2)
    fig.update_yaxes(title_text="Residuals", row=2, col=1)
    fig.update_yaxes(title_text="MAE", row=2, col=2)
    
    fig.show()
    
    # Additional Analysis: Directional Accuracy
    if 'Ensemble' in backtest_forecasts.columns and len(backtest_forecasts) > 1:
        print("\n=== DIRECTIONAL ACCURACY ANALYSIS ===")
        
        # Calculate month-over-month changes
        actual_changes = backtest_forecasts['Actual'].diff().dropna()
        forecast_changes = backtest_forecasts['Ensemble'].diff().dropna()
        
        # Calculate directional accuracy (same sign of change)
        directional_accuracy = np.mean((actual_changes * forecast_changes) > 0) * 100
        
        print(f"Directional Accuracy: {directional_accuracy:.1f}%")
        print("(Percentage of time forecast correctly predicted direction of change)")
        
        # Additional directional analysis
        correct_direction = (actual_changes * forecast_changes) > 0
        print(f"\nDirectional Analysis:")
        print(f"  Months with correct direction: {correct_direction.sum()}/{len(correct_direction)}")
        print(f"  Months with incorrect direction: {(~correct_direction).sum()}/{len(correct_direction)}")
        
        # Show specific months with wrong direction
        if (~correct_direction).any():
            wrong_dates = backtest_forecasts['Date'].iloc[1:][~correct_direction]
            print(f"  Months with wrong direction: {', '.join(wrong_dates.dt.strftime('%Y-%m'))}")
        
        # Normality test on residuals
        if len(residuals) > 3:
            from scipy import stats
            try:
                shapiro_stat, shapiro_p = stats.shapiro(residuals)
                print(f"\nResiduals Normality Test (Shapiro-Wilk):")
                print(f"  Test Statistic: {shapiro_stat:.4f}")
                print(f"  P-value: {shapiro_p:.4f}")
                normality_result = "Normal" if shapiro_p > 0.05 else "Not Normal"
                print(f"  Result: {normality_result} at 5% significance level")
            except:
                print(f"\nResiduals Normality Test: Could not be performed")
        
        # Mean Absolute Deviation
        mad = np.mean(np.abs(residuals))
        print(f"\nMean Absolute Deviation: {mad:,.0f}")
        
        # Forecast bias analysis
        bias = np.mean(residuals)
        print(f"Forecast Bias: {bias:,.0f}")
        if bias > 0:
            print("  Interpretation: Forecast tends to underpredict")
        elif bias < 0:
            print("  Interpretation: Forecast tends to overpredict")
        else:
            print("  Interpretation: Forecast is unbiased")
        
        # Percentage bias
        bias_pct = (bias / np.mean(backtest_forecasts['Actual'])) * 100
        print(f"Percentage Bias: {bias_pct:.2f}%")
    
    print("\n=== BACKTESTING VISUALIZATION COMPLETE ===")
    
else:
    print("⚠️ Skipping backtesting visualization - data unavailable")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 181, Finished, Available, Finished)


=== BACKTESTING VISUALIZATION ===



=== DIRECTIONAL ACCURACY ANALYSIS ===
Directional Accuracy: 81.8%
(Percentage of time forecast correctly predicted direction of change)

Directional Analysis:
  Months with correct direction: 9/11
  Months with incorrect direction: 2/11
  Months with wrong direction: 2024-05, 2024-10

Residuals Normality Test (Shapiro-Wilk):
  Test Statistic: 0.8683
  P-value: 0.0623
  Result: Normal at 5% significance level

Mean Absolute Deviation: 2,663,239
Forecast Bias: 1,269,315
  Interpretation: Forecast tends to underpredict
Percentage Bias: 5.20%

=== BACKTESTING VISUALIZATION COMPLETE ===


### Backtesting Summary Report

In [180]:
# Generate Comprehensive Backtesting Summary Report
if len(data_test) > 0:
    print("\n=== GENERATING BACKTESTING SUMMARY REPORT ===")
    
    # Create comprehensive summary report
    summary_report_data = []
    
    # Overall performance summary
    summary_report_data.append({
        'Section': 'Overall Performance',
        'Metric': 'Best Method',
        'Value': best_method,
        'Details': f'MAPE: {best_mape:.2f}%'
    })
    
    # Add detailed metrics for best method
    best_method_metrics = accuracy_df[accuracy_df['Method'] == best_method].iloc[0]
    
    key_metrics = [
        ('MAE', 'Mean Absolute Error', f"{best_method_metrics['MAE']:,.0f}"),
        ('RMSE', 'Root Mean Square Error', f"{best_method_metrics['RMSE']:,.0f}"),
        ('MAPE', 'Mean Absolute Percentage Error', f"{best_method_metrics['MAPE']:.2f}%"),
        ('R2', 'R-Squared', f"{best_method_metrics['R2']:.4f}"),
        ('Bias', 'Forecast Bias', f"{best_method_metrics['Bias']:,.0f}"),
        ('Bias_Percent', 'Bias Percentage', f"{best_method_metrics['Bias_Percent']:.2f}%")
    ]
    
    for metric_code, metric_name, value in key_metrics:
        summary_report_data.append({
            'Section': 'Best Method Metrics',
            'Metric': metric_name,
            'Value': value,
            'Details': f'{best_method} performance'
        })
    
    # Data coverage
    summary_report_data.append({
        'Section': 'Data Coverage',
        'Metric': 'Training Period',
        'Value': f"{data_train['Date'].min().strftime('%Y-%m')} to {data_train['Date'].max().strftime('%Y-%m')}",
        'Details': f"{len(data_train)} months"
    })
    
    summary_report_data.append({
        'Section': 'Data Coverage',
        'Metric': 'Test Period',
        'Value': f"{data_test['Date'].min().strftime('%Y-%m')} to {data_test['Date'].max().strftime('%Y-%m')}",
        'Details': f"{len(overall_actual)} months"
    })
    
    # Model comparison insights
    if len(accuracy_results) > 1:
        worst_method = accuracy_df.loc[accuracy_df['MAPE'].idxmax(), 'Method']
        worst_mape = accuracy_df.loc[accuracy_df['MAPE'].idxmax(), 'MAPE']
        improvement = worst_mape - best_mape
        
        summary_report_data.append({
            'Section': 'Model Insights',
            'Metric': 'Method Improvement',
            'Value': f"{improvement:.2f}% MAPE reduction",
            'Details': f"{best_method} vs {worst_method}"
        })
        
        # Add all method performances
        for result in accuracy_results:
            summary_report_data.append({
                'Section': 'Method Performance',
                'Metric': f"{result['Method']} MAPE",
                'Value': f"{result['MAPE']:.2f}%",
                'Details': f"MAE: {result['MAE']:,.0f}, R²: {result['R2']:.4f}"
            })
    
    # Statistical significance
    if 'Ensemble' in backtest_forecasts.columns:
        residuals = backtest_forecasts['Actual'] - backtest_forecasts['Ensemble']
        
        # Shapiro-Wilk test for normality of residuals
        try:
            from scipy import stats
            shapiro_stat, shapiro_p = stats.shapiro(residuals)
            normality_result = "Normal" if shapiro_p > 0.05 else "Non-normal"
            
            summary_report_data.append({
                'Section': 'Statistical Tests',
                'Metric': 'Residuals Normality',
                'Value': normality_result,
                'Details': f"Shapiro-Wilk p-value: {shapiro_p:.4f}"
            })
        except:
            summary_report_data.append({
                'Section': 'Statistical Tests',
                'Metric': 'Residuals Normality',
                'Value': "Could not test",
                'Details': "Test not available"
            })
        
        # Mean absolute deviation
        mad = np.mean(np.abs(residuals - np.mean(residuals)))
        summary_report_data.append({
            'Section': 'Statistical Tests',
            'Metric': 'Mean Absolute Deviation',
            'Value': f"{mad:,.0f}",
            'Details': "Residuals spread measure"
        })
        
        # Directional accuracy
        if len(backtest_forecasts) > 1:
            actual_changes = backtest_forecasts['Actual'].diff().dropna()
            forecast_changes = backtest_forecasts['Ensemble'].diff().dropna()
            directional_accuracy = np.mean((actual_changes * forecast_changes) > 0) * 100
            
            summary_report_data.append({
                'Section': 'Directional Analysis',
                'Metric': 'Directional Accuracy',
                'Value': f"{directional_accuracy:.1f}%",
                'Details': "Correct direction prediction rate"
            })
    
    # Forecast characteristics
    summary_report_data.append({
        'Section': 'Forecast Characteristics',
        'Metric': 'Training Data Mean',
        'Value': f"{data_train['Quantity Invoiced'].mean():,.0f}",
        'Details': "Average monthly quantity in training"
    })
    
    summary_report_data.append({
        'Section': 'Forecast Characteristics',
        'Metric': 'Test Data Mean',
        'Value': f"{overall_actual['Quantity Invoiced'].mean():,.0f}",
        'Details': "Average monthly quantity in test period"
    })
    
    if 'Ensemble' in backtest_forecasts.columns:
        summary_report_data.append({
            'Section': 'Forecast Characteristics',
            'Metric': 'Ensemble Forecast Mean',
            'Value': f"{backtest_forecasts['Ensemble'].mean():,.0f}",
            'Details': "Average monthly forecast"
        })
    
    # Create summary DataFrame
    summary_df_report = pd.DataFrame(summary_report_data)
    
    # Export summary report
    summary_df_report.to_csv(data_location + 'modelGeneratedData/backtesting_summary_report.csv', index=False)

    # Create a csv with the actuals and overall forecast of each method for each month
    export_df = backtest_forecasts.copy()
    
    # Clean up columns for export
    export_columns = ['Date', 'Actual']
    for method in ['ARIMA', 'SARIMA', 'ExpSmoothing', 'Prophet', 'Ensemble']:
        if method in export_df.columns:
            export_columns.append(method)
    
    export_df = export_df[export_columns]
    export_df.to_csv(data_location + 'backtesting_actuals_vs_forecast_by_method.csv', index=False)
    print('Exported actuals and forecasts by method to: backtesting_actuals_vs_forecast_by_method.csv')

    # Display formatted summary
    print(f"\n{'='*80}")
    print(f"                 OVERALL FORECASTING BACKTESTING SUMMARY")
    print(f"{'='*80}")
    
    current_section = ""
    for _, row in summary_df_report.iterrows():
        if row['Section'] != current_section:
            current_section = row['Section']
            print(f"\n🔹 {current_section.upper()}")
            print(f"{'-'*50}")
        
        print(f"  {row['Metric']:<30}: {row['Value']:<20} ({row['Details']})")
    
    print(f"\n{'='*80}")
    print(f"📊 Complete backtesting summary exported to: backtesting_summary_report.csv")
    print(f"🎯 Overall forecasting backtesting analysis completed successfully!")
    
    # Display final summary table
    print(f"\n📋 FINAL BACKTESTING RESULTS:")
    summary_table = accuracy_df[['Method', 'MAPE', 'MAE', 'R2', 'Bias_Percent']].round(2)
    print(summary_table.to_string(index=False))
    
    # Key insights
    print(f"\n🔍 KEY INSIGHTS:")
    print(f"  • Best performing method: {best_method} with {best_mape:.2f}% MAPE")
    print(f"  • Test period covered: {len(overall_actual)} months")
    print(f"  • Training period: {len(data_train)} months")
    if 'Ensemble' in backtest_forecasts.columns:
        ensemble_bias = np.mean(backtest_forecasts['Actual'] - backtest_forecasts['Ensemble'])
        bias_direction = "under-predicts" if ensemble_bias > 0 else "over-predicts" if ensemble_bias < 0 else "is unbiased"
        print(f"  • Ensemble forecast {bias_direction} by {abs(ensemble_bias):,.0f} units on average")
    
    display(summary_df_report.head(15))
else:
    print("⚠️ Summary report generation skipped - data unavailable")

StatementMeta(, af3036fc-66ac-43d4-ba33-58567deb700b, 182, Finished, Available, Finished)


=== GENERATING BACKTESTING SUMMARY REPORT ===
Exported actuals and forecasts by method to: backtesting_actuals_vs_forecast_by_method.csv

                 OVERALL FORECASTING BACKTESTING SUMMARY

🔹 OVERALL PERFORMANCE
--------------------------------------------------
  Best Method                   : Exponential Smoothing (MAPE: 8.81%)

🔹 BEST METHOD METRICS
--------------------------------------------------
  Mean Absolute Error           : 2,010,660            (Exponential Smoothing performance)
  Root Mean Square Error        : 2,617,345            (Exponential Smoothing performance)
  Mean Absolute Percentage Error: 8.81%                (Exponential Smoothing performance)
  R-Squared                     : 0.4504               (Exponential Smoothing performance)
  Forecast Bias                 : 94,803               (Exponential Smoothing performance)
  Bias Percentage               : 0.39%                (Exponential Smoothing performance)

🔹 DATA COVERAGE
-----------------------

SynapseWidget(Synapse.DataFrame, 670c0649-e735-4efc-9936-f011d79c0bfa)